In [217]:
#Arabidopsis  RNA-seq and protein level data for 30 different tissues was taken from Mergner et al 2020 https://www.nature.com/articles/s41586-020-2094-2
#the 2nd sheet (gene expression) of supplementary table 2 was saved as a csv. Location: ../../data/genes/mergner2020_RNA-seq_CVs.csv

#this notebook calculates the mean and SD and then CV (coefficient of variation, SD/mean) allowing ranking of genes by how stably expressed they are

##can us https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.variation.html for calculating cv in future

In [227]:
import pandas as pd

In [228]:
def calculate_cv(df, data_type):
    """function to calculate the cv (SD/mean) of the RNA or protein_df. Provide the dataframe containing columns of interest."""

    #reverse log transform
    df2 = df.copy()
    df2.iloc[:,3:] = df.iloc[:,3:].apply(lambda x: 2**x)
    #change NaN values to 0 expression (mRNA quantities are displayed as TPM and a cut-off value of 1 TPM was used as lower limit of detection across all samples)
    df = df2.copy()
    df.iloc[:,3:] = df2.iloc[:,3:].fillna(0)

    #calculate mean
    df = df.assign(mean=df.iloc[:,3:].mean(axis=1))
    df.rename(columns={'mean':f'{data_type}_mean'}, inplace=True)
    #calculate SD
    df = df.assign(sd=df.iloc[:,3:].std(axis=1))
    df.rename(columns={'sd':f'{data_type}_sd'}, inplace=True)
    #calculate CV
    df = df.assign(cv=df[f'{data_type}_sd'] / df[f'{data_type}_mean'])
    df.rename(columns={'cv':f'{data_type}_cv'}, inplace=True)
    
    return df
    

In [229]:
def save_csv(input_df, output):
    """function to extract useful columns and save them as a csv"""
    df = input_df.iloc[:, [0,1,2,35]]
    #remove nan
    df2 = df[df.iloc[:,3].notnull()]
    df_sorted = df2.sort_values(df.columns[3]).reset_index(drop=True)
    df_sorted.to_csv(output, index=False)
    
    

In [230]:
original_csv = '../../data/genes/mergner2020_RNA-seq_CVs.csv'
df = pd.read_csv(csv, header=1)
RNA_df = df[['AGI code', 'transcription (iTAK)_class', 'transcription (iTAK)_family', 'TPM_SP', 'TPM_PT','TPM_ST','TPM_CP','TPM_SQ','TPM_EB','TPM_SD','TPM_SDIMB','TPM_P','TPM_ND','TPM_IND','TPM_CLLF','TPM_LFD','TPM_LFP','TPM_LFPT','TPM_SCLF','TPM_TPM_RT','TPM_FLPD','TPM_FL','TPM_SQSP',
             'TPM_SQV','TPM_RKD2','TPM_CIM','TPM_CC3','TPM_CC10','TPM_CT','TPM_HY','TPM_RTTP','TPM_RTUZ','TPM_CTSAM'
]]
protein_df = df[['AGI code', 'transcription (iTAK)_class', 'transcription (iTAK)_family','iBAQ_SP','iBAQ_PT','iBAQ_ST','iBAQ_CP','iBAQ_SQ','iBAQ_EB','iBAQ_SD','iBAQ_SDIMB',
                 'iBAQ_P','iBAQ_ND','iBAQ_IND','iBAQ_CLLF','iBAQ_LFD','iBAQ_LFP','iBAQ_LFPT','iBAQ_SCLF','iBAQ_iBAQ_RT','iBAQ_FLPD','iBAQ_FL','iBAQ_SQSP','iBAQ_SQV','iBAQ_RKD2','iBAQ_CIM',
                 'iBAQ_CC3','iBAQ_CC10','iBAQ_CT','iBAQ_HY','iBAQ_RTTP','iBAQ_RTUZ','iBAQ_CTSAM'
]]

In [231]:
RNA_output = '../../data/genes/RNA_CVs.csv'
protein_output = '../../data/genes/proteincontent_CVs.csv'

In [232]:
rna_reversetransformed = calculate_cv(RNA_df, 'RNA')
protein_reersetransformed = calculate_cv(protein_df, 'proteincontent')

In [233]:
save_csv(rna_reversetransformed, RNA_output)
save_csv(protein_reersetransformed, protein_output)

In [234]:
rna_reversetransformed

,AGI code,transcription (iTAK)_class,transcription (iTAK)_family,TPM_SP,TPM_PT,TPM_ST,TPM_CP,TPM_SQ,TPM_EB,TPM_SD,...,TPM_CC3,TPM_CC10,TPM_CT,TPM_HY,TPM_RTTP,TPM_RTUZ,TPM_CTSAM,RNA_mean,RNA_sd,RNA_cv
0,AT1G01010,TF,NAC,11.704002,1.647328,4.036626,8.957710,2.381351,1.882513,7.247583,...,12.640135,38.439029,3.698371,7.469543,29.875273,27.059439,2.856246,12.455559,11.780531,0.945805
1,AT1G01020,NaN,NaN,9.283636,11.689084,10.586766,12.057342,11.559443,10.061894,5.715343,...,3.942717,8.223029,6.991248,6.876955,5.345074,8.647901,8.319461,8.895660,3.422566,0.384746
2,AT1G01030,TF,B3,7.803330,8.797478,0.000000,2.566834,2.137093,1.309977,1.853502,...,0.000000,0.000000,5.445332,0.000000,0.000000,1.471436,4.723545,2.203977,2.412849,1.094770
3,AT1G01040,NaN,NaN,21.664935,24.035751,15.250557,17.991705,12.463696,2.069511,5.967348,...,10.859155,10.466738,12.387129,12.238629,11.636054,11.501260,12.517287,14.592543,5.677111,0.389042
4,AT1G01050,NaN,NaN,79.752067,68.938595,95.039807,64.071906,103.273049,137.962274,55.346877,...,36.635490,45.241034,87.176054,89.479792,108.970967,101.138607,89.196179,86.871489,26.289480,0.302625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25280,ATMG01350,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,3.554976,2.974568,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.124098,3.535268,3.144983
25281,ATMG01360,NaN,NaN,6.850551,2.534340,12.569366,1.137180,2.985453,120.133108,38.998859,...,4.035144,1.899880,0.000000,3.179170,0.000000,0.000000,0.000000,17.837329,43.896640,2.460942
25282,ATMG01370,NaN,NaN,21.724785,5.600051,3.074177,10.175798,11.817640,305.176487,36.483191,...,2.081452,4.779137,6.972616,18.518046,0.000000,1.445568,3.305985,21.242845,53.853479,2.535135
25283,ATMG01400,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.472564,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.049085,0.264333,5.385165


In [235]:
protein_reersetransformed

,AGI code,transcription (iTAK)_class,transcription (iTAK)_family,iBAQ_SP,iBAQ_PT,iBAQ_ST,iBAQ_CP,iBAQ_SQ,iBAQ_EB,iBAQ_SD,...,iBAQ_CC3,iBAQ_CC10,iBAQ_CT,iBAQ_HY,iBAQ_RTTP,iBAQ_RTUZ,iBAQ_CTSAM,proteincontent_mean,proteincontent_sd,proteincontent_cv
0,AT1G01010,TF,NAC,2.215542e+05,0.000000e+00,3.270319e+06,1.578287e+05,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,6.755954e+05,0.000000e+00,0.000000e+00,1.693880e+06,9.396083e+05,0.000000e+00,4.466096e+05,7.622711e+05,1.706795
1,AT1G01020,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
2,AT1G01030,TF,B3,1.205416e+06,2.907605e+06,1.677787e+05,1.027709e+06,7.518055e+05,2.134591e+05,0.000000e+00,...,0.000000e+00,0.000000e+00,4.658158e+05,0.000000e+00,3.112253e+05,0.000000e+00,1.117389e+06,3.349197e+05,6.204351e+05,1.852489
3,AT1G01040,NaN,NaN,1.738881e+05,1.190305e+06,6.004148e+05,6.577592e+05,5.272033e+05,4.609018e+05,2.409376e+05,...,3.509174e+06,7.077597e+05,3.730217e+05,2.896598e+05,1.144432e+06,6.138387e+05,7.597683e+05,6.187560e+05,6.275489e+05,1.014211
4,AT1G01050,NaN,NaN,3.084275e+06,2.252351e+06,2.879526e+06,7.547294e+05,1.386100e+06,2.009214e+06,0.000000e+00,...,0.000000e+00,5.216678e+06,0.000000e+00,4.430076e+06,3.252235e+06,0.000000e+00,5.085295e+06,2.796509e+06,2.448531e+06,0.875567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25280,ATMG01350,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
25281,ATMG01360,NaN,NaN,4.464515e+07,5.333946e+07,1.087319e+08,3.601514e+07,4.733487e+07,2.196495e+07,1.357642e+07,...,6.560034e+07,3.240246e+07,1.693847e+07,3.733176e+07,3.720002e+07,6.919670e+07,1.538691e+07,4.064264e+07,3.757992e+07,0.924643
25282,ATMG01370,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
25283,ATMG01400,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
